In [ ]:
#test 13 man VS photo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
#import pickle
#import zipfile
#import csv
import sys
import os
#import math


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import *
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB3, EfficientNetB6

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
from keras.models import Sequential
#print(os.listdir("../input/sf-dl-car-classification"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [ ]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.
RANDOM_SEED          = 100

EPOCHS               = 5  # эпох на обучение
BATCH_SIZE           = 64 # test2 -128
LR                   = 0.0001
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 3  
IMG_SIZE             = 224 # какого размера подаем изображения в сеть
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)

PATH = "../input/nnnew-ds/" # рабочая директория

In [ ]:
image = PIL.Image.open(PATH + '0/0.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

In [ ]:
#Аугментация данных
train_datagen = ImageDataGenerator(
    rescale=1. / 255, #коэффициент масштабирования. По умолчанию - Нет. Если Нет или 0, 
    #масштабирование не применяется, в противном случае мы умножаем данные на предоставленное значение 
    #(после применения всех остальных преобразований).
    
    rotation_range = 5, #угол случайных поворотов
    width_shift_range = 0.1, #диапазон сдвига ширины
    height_shift_range = 0.1, #диапазон сдвига высоты
    #brightness_range =  [0.5, 1.5], # сдвиг яркости
    validation_split = VAL_SPLIT, # Доля выборки для валидации 
    horizontal_flip = True  # отражение по горизонтали
    )

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
#Генерация данных
train_generator = train_datagen.flow_from_directory(
    #PATH,#+'train/',      # директория где расположены папки с картинками 
    '../input/nnnew-ds/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training') # set as training data

test_generator = train_datagen.flow_from_directory(
    #PATH,#+'train/',
    '../input/nnnew-ds/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation') # set as validation data

In [ ]:
#Строим модель
#Загружаем предобученную сеть Xception:
base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

#base_model.trainable = False
#base_model.summary()
#Устанавливаем новую «голову» (head):
x = base_model.output
x = GlobalAveragePooling2D()(x)  #Pooling слой
x = BatchNormalization()(x) #добавим Batch нормализацию
x = Dense(256, activation='elu')(x) # полносвязный слой с активацией elu
x = Dropout(0.25)(x) # полносвязный слой с вероятность отключения нейронов в слое
predictions = Dense(CLASS_NUM, activation='softmax')(x) #логистический слой c 3 классами


model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])


In [ ]:
#Добавим ModelCheckpoint. Эта функция позволяет сохранять прогресс обучения модели,
#чтобы в нужный момент можно было его подгрузить и дообучить модель.
checkpoint = ModelCheckpoint('test13.hdf5' , monitor = ['val_accuracy'] , verbose = 1  , mode = 'max')

#останоить обучение, когда метрика перестает улучшаться.
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 4, restore_best_weights = True)

#Снижайть скорость обучения, когда метрика перестает улучшаться.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=2, min_lr=0.0000001, verbose=1, mode='auto')
    
callbacks_list = [checkpoint, earlystop, reduce_lr]

In [ ]:
base_model.trainable = True

In [ ]:
model.load_weights('../input/test12-0/test12(3).hdf5')

In [ ]:
#Обучаем модель
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = 25, #EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
#Обучаем модель
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = 25, #EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
#Обучаем модель
history = model.fit_generator(
        train_generator,
        steps_per_epoch = len(train_generator),
        validation_data = test_generator, 
        validation_steps = len(test_generator),
        epochs = 25, #EPOCHS,
        callbacks = callbacks_list
)

In [ ]:
from IPython.display import FileLink
FileLink(r'./test13.hdf5')

In [ ]:
import cv2

In [ ]:
mix_img = cv2.imread('../input/new-dataset/0/0.png')
image = cv2.resize(mix_img[..., ::-1], (224, 224) )  
#image  = mix_img 
image2 = image / 255 
pred = model.predict(image[None, ...])

In [ ]:
imgplot = plt.imshow(image)
plt.show()


In [ ]:
image = cv2.imread('../input/new-dataset/2/p0 (10).png')[..., ::-1]
image3 = cv2.resize(image, (224, 224))/255
pred = model.predict(image3[None, ...])
print(pred)
predictions = np.argmax(pred, axis=-1)
print(predictions)
